In [23]:
import torch
import pandas as pd
import tarfile
import os
import matplotlib.pyplot as plt

In [24]:
if torch.cuda.is_available():
    device = torch.device('cuda')
    print("CUDA is available. Using GPU.")
else:
    device = torch.device('cpu')
    print("CUDA is not available. Using CPU.")

CUDA is available. Using GPU.


In [25]:
patient_information = pd.read_csv('EPIC_EMR/EMR/patient_information.csv')
patient_information = patient_information.drop_duplicates()
hours_df = pd.read_csv('hours_fb_df_new.csv', index_col=0)

In [26]:
def agg_func(series):
    if series.nunique() == 1:
        return series.iloc[0]
    else:
        return '|'.join(map(str, series.unique()))

In [27]:
patient_information['HOSP_ADMSN_TIME'] = pd.to_datetime(patient_information['HOSP_ADMSN_TIME'])
patient_information['HOSP_DISCH_TIME'] = pd.to_datetime(patient_information['HOSP_DISCH_TIME'])
hours_df['HOSP_ADMSN_TIME'] = pd.to_datetime(hours_df['HOSP_ADMSN_TIME'])
hours_df['HOSP_DISCH_TIME'] = pd.to_datetime(hours_df['HOSP_DISCH_TIME'])

/tmp/ipykernel_129265/3590612282.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  patient_information['HOSP_ADMSN_TIME'] = pd.to_datetime(patient_information['HOSP_ADMSN_TIME'])
/tmp/ipykernel_129265/3590612282.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  patient_information['HOSP_DISCH_TIME'] = pd.to_datetime(patient_information['HOSP_DISCH_TIME'])


In [28]:
patient_information.columns

Index(['LOG_ID', 'MRN', 'DISCH_DISP_C', 'DISCH_DISP', 'HOSP_ADMSN_TIME',
       'HOSP_DISCH_TIME', 'LOS', 'ICU_ADMIN_FLAG', 'SURGERY_DATE',
       'BIRTH_DATE', 'HEIGHT', 'WEIGHT', 'SEX', 'PRIMARY_ANES_TYPE_NM',
       'ASA_RATING_C', 'ASA_RATING', 'PATIENT_CLASS_GROUP', 'PATIENT_CLASS_NM',
       'PRIMARY_PROCEDURE_NM', 'IN_OR_DTTM', 'OUT_OR_DTTM',
       'AN_START_DATETIME', 'AN_STOP_DATETIME'],
      dtype='object')

In [29]:
# Merge demographic data
cols = ['MRN', 'LOG_ID', 'ICU_ADMIN_FLAG', 'SURGERY_DATE', 'DISCH_DISP_C', 'DISCH_DISP', 'BIRTH_DATE', 'SEX', 'PRIMARY_PROCEDURE_NM', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'ASA_RATING_C', 'ASA_RATING']
final_df = pd.merge(hours_df, patient_information[cols], on=['MRN', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME'], how='inner')
final_df

,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,HOURS,LOG_ID,ICU_ADMIN_FLAG,SURGERY_DATE,DISCH_DISP_C,DISCH_DISP,BIRTH_DATE,SEX,PRIMARY_PROCEDURE_NM,ASA_RATING_C,ASA_RATING
0,38f2abdfacbed586,2021-08-24 07:19:00,2021-09-06 14:56:00,0.0,2e9484ebef54f1b2,Yes,8/24/21 0:00,6.0,Skilled Nursing Facility,82,Male,IR INSERT CHOLECYSTOSTOMY TUBE,NaN,NaN
1,38f2abdfacbed586,2021-08-24 07:19:00,2021-09-06 14:56:00,0.0,ce4f955d458acfdd,Yes,8/26/21 0:00,6.0,Skilled Nursing Facility,82,Male,IR INSERT CHOLECYSTOSTOMY TUBE,NaN,NaN
2,38f2abdfacbed586,2021-08-24 07:19:00,2021-09-06 14:56:00,0.0,f84a2bf4925a9b13,Yes,9/2/21 0:00,6.0,Skilled Nursing Facility,82,Male,"EGD, WITH BIOPSY",3.0,Severe Systemic Disease
3,2d604e5cc7818233,2022-08-08 22:20:00,2022-08-23 15:28:00,0.0,e9fa1fab0d8922fb,Yes,8/9/22 0:00,20.0,Home Healthcare IP Admit Related,42,Male,CRANIECTOMY,3.0,Severe Systemic Disease
4,3b7fd5bd48dae180,2022-01-20 01:47:00,2022-01-31 15:39:00,0.0,8dc848e496b34760,Yes,1/26/22 0:00,20.0,Home Healthcare IP Admit Related,66,Male,"REPLACEMENT, MITRAL VALVE",4.0,Incapacitating Disease
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6015,5800610092b18862,2020-01-31 19:26:00,2020-03-10 11:25:00,0.0,5b2468b0f5162c9f,Yes,2/21/20 0:00,16.0,Hospice Facility,61,Male,THORACENTESIS,NaN,NaN
6016,5800610092b18862,2020-01-31 19:26:00,2020-03-10 11:25:00,0.0,c50dc1b7e703bfd8,Yes,2/27/20 0:00,16.0,Hospice Facility,61,Male,EGD (ESOPHAGOGASTRODUODENOSCOPY),NaN,NaN
6017,5800610092b18862,2020-01-31 19:26:00,2020-03-10 11:25:00,0.0,8109c254e2c57519,Yes,2/28/20 0:00,16.0,Hospice Facility,61,Male,EGD (ESOPHAGOGASTRODUODENOSCOPY),4.0,Incapacitating Disease
6018,5800610092b18862,2020-01-31 19:26:00,2020-03-10 11:25:00,0.0,bf0be14ec26f19d3,Yes,3/4/20 0:00,16.0,Hospice Facility,61,Male,EGD (ESOPHAGOGASTRODUODENOSCOPY),3.0,Severe Systemic Disease


In [30]:
# Merge diagnoses and post-op complications
post_op_complications = pd.read_csv('EPIC_EMR/EMR/patient_post_op_complications.csv')
patient_visit = pd.read_csv('EPIC_EMR/EMR/patient_visit.csv')
patient_visit = patient_visit.drop_duplicates()
patient_visit.rename(columns={'mrn': 'MRN'}, inplace=True)
diagnosis_and_complications = pd.merge(patient_visit[['LOG_ID', 'MRN', 'diagnosis_code', 'dx_name']],
                                       post_op_complications[['LOG_ID', 'MRN', 'Element_abbr']],
                                       on=['LOG_ID', 'MRN'],
                                       how='inner')
diagnosis_and_complications = diagnosis_and_complications.drop_duplicates()
diagnosis_and_complications_final = diagnosis_and_complications.groupby(['MRN', 'LOG_ID']).agg(agg_func).reset_index()
diagnosis_and_complications_final

,MRN,LOG_ID,diagnosis_code,dx_name,Element_abbr
0,0000c7ccaa009343,47fbfb9938d6067b,592.0,Nephrolithiasis|Kidney stone,AN Post-op Complications
1,0000c7ccaa009343,bd16a67880da1e77,789.00,Left flank pain,AN Post-op Complications
2,0000e45237d1fc96,611e9c50425dbf55,nan,"Gunshot wound of right lower extremity, initia...",AN Post-op Complications
3,000181ce0ebf1e12,be9a505e236e5b67,193,Papillary thyroid carcinoma (CMS-HCC),AN Post-op Complications
4,00018b4d9acb258b,189a1d9df810fe26,307.9|nan|786.05|786.09|790.4,"Agitation|Impaired functional mobility, balanc...",AN Post-op Complications
...,...,...,...,...,...
46248,fff634f64e07b66a,98183e21fcda44c9,585.6,ESRD (end stage renal disease) on dialysis (CM...,AN Post-op Complications
46249,fffb5d2e00cf670f,ca86eca1473b99bf,nan,"Ruptured globe of right eye, initial encounter",AN Post-op Complications
46250,fffc39a39b57eee7,f18c7d34c4192c9b,nan,"Pseudoaneurysm of AV hemodialysis fistula, ini...",AN Post-op Complications
46251,fffcb59d973c7e4c,c293e9fef7b0f159,427.89,SVT (supraventricular tachycardia) (CMS-HCC),AN Post-op Complications


In [31]:
# Merge all data
final_df_merged = pd.merge(final_df, diagnosis_and_complications_final, on=['LOG_ID', 'MRN'], how='inner')
final_df_merged

,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,HOURS,LOG_ID,ICU_ADMIN_FLAG,SURGERY_DATE,DISCH_DISP_C,DISCH_DISP,BIRTH_DATE,SEX,PRIMARY_PROCEDURE_NM,ASA_RATING_C,ASA_RATING,diagnosis_code,dx_name,Element_abbr
0,38f2abdfacbed586,2021-08-24 07:19:00,2021-09-06 14:56:00,0.000000,f84a2bf4925a9b13,Yes,9/2/21 0:00,6.0,Skilled Nursing Facility,82,Male,"EGD, WITH BIOPSY",3.0,Severe Systemic Disease,578.1|785.59,Melena|Hemorrhagic shock (CMS-HCC),AN Post-op Complications
1,2d604e5cc7818233,2022-08-08 22:20:00,2022-08-23 15:28:00,0.000000,e9fa1fab0d8922fb,Yes,8/9/22 0:00,20.0,Home Healthcare IP Admit Related,42,Male,CRANIECTOMY,3.0,Severe Systemic Disease,324.9|nan|780.39,Cranial epidural abscess|Nontraumatic subcorti...,AN Post-op Complications|Administrative
2,3b7fd5bd48dae180,2022-01-20 01:47:00,2022-01-31 15:39:00,0.000000,8dc848e496b34760,Yes,1/26/22 0:00,20.0,Home Healthcare IP Admit Related,66,Male,"REPLACEMENT, MITRAL VALVE",4.0,Incapacitating Disease,394.0,Mitral stenosis,AN Post-op Complications
3,0ea0ab0b01b01c4c,2019-01-12 18:19:00,2019-02-07 18:14:00,0.000000,8dd494b84151815e,Yes,1/13/19 0:00,6.0,Skilled Nursing Facility,51,Male,"CRANIOTOMY, FOR ANEURYSM OR AVM CLIPPING, WITH...",5.0,Moribund,431|348.5|429.83|401.9|785.50|432.9|430|nan|79...,IVH (intraventricular hemorrhage) (CMS-HCC)|Ce...,AN Post-op Complications
4,bba97a9988e7a2aa,2019-03-24 04:07:00,2019-06-22 18:50:00,0.000000,757c2bee63380c85,Yes,4/2/19 0:00,26.0,Acute Care Facility (not this hospital),43,Female,ORIF ANTERIOR PELVIS,4.0,Incapacitating Disease,442.3|854.00|432.1|nan,"Femoral artery pseudo-aneurysm, right (CMS-HCC...",AN Post-op Complications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2175,bb2936abd9e070d7,2021-10-21 15:06:00,2021-10-26 00:59:00,0.037777,c2f69611af88a940,Yes,10/21/21 0:00,16.0,Hospice Facility,53,Male,"CRANIECTOMY OR CRANIOTOMY, EMERGENT",4.0,Incapacitating Disease,432.0|444.1,Epidural hematoma|Aortic mural thrombus (CMS-HCC),AN Post-op Complications
2176,224c476538275696,2022-06-04 22:17:00,2022-07-17 21:59:00,0.000000,90eafc136cb54259,Yes,6/7/22 0:00,107.0,Sub-Acute Care Facility,45,Female,"ORIF, FRACTURE, FEMUR",4.0,Incapacitating Disease,828.0|263.0|998.32|959.9|nan|117.9,Multiple extremity fractures|Moderate protein-...,AN Post-op Complications
2177,5800610092b18862,2019-09-22 07:41:00,2019-11-24 17:05:00,0.540456,a3afe6ae9e8d71d9,Yes,11/10/19 0:00,30.0,Long Term Care Facility,60,Male,EGD (ESOPHAGOGASTRODUODENOSCOPY),3.0,Severe Systemic Disease,V49.89|V45.89|157.9|250.00|117.9|442.9|nan,On total parenteral nutrition (TPN)|Status pos...,AN Post-op Complications|Administrative|Airway
2178,5800610092b18862,2020-01-31 19:26:00,2020-03-10 11:25:00,0.000000,bf0be14ec26f19d3,Yes,3/4/20 0:00,16.0,Hospice Facility,61,Male,EGD (ESOPHAGOGASTRODUODENOSCOPY),3.0,Severe Systemic Disease,nan|157.1|783.7,"Non-intractable vomiting with nausea, unspecif...",Administrative|AN Post-op Complications|Airway


In [32]:
collapsed_df_final = final_df_merged.groupby(['MRN', 'HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME']).agg(agg_func).reset_index()
collapsed_df_final.to_csv('all_data_final.csv')
collapsed_df_final

,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,HOURS,LOG_ID,ICU_ADMIN_FLAG,SURGERY_DATE,DISCH_DISP_C,DISCH_DISP,BIRTH_DATE,SEX,PRIMARY_PROCEDURE_NM,ASA_RATING_C,ASA_RATING,diagnosis_code,dx_name,Element_abbr
0,00018b4d9acb258b,2020-07-25 13:33:00,2020-08-25 16:15:00,0.000000,189a1d9df810fe26,Yes,8/3/20 0:00,6.0,Skilled Nursing Facility,80,Male,"BIOPSY, MUSCLE",4.0,Incapacitating Disease,307.9|nan|786.05|786.09|790.4,"Agitation|Impaired functional mobility, balanc...",AN Post-op Complications
1,000db5bb4b440912,2021-10-21 07:53:00,2021-10-28 15:33:00,0.000000,21d28a9add40b0c4,Yes,10/23/21 0:00,20.0,Home Healthcare IP Admit Related,77,Male,CABG (CORONARY ARTERY BYPASS GRAFT),4.0,Incapacitating Disease,794.39,Abnormal stress test,AN Post-op Complications
2,000f8e3ae0b5aa91,2020-05-27 05:58:00,2020-05-27 16:00:00,0.000000,80afb4c2735257cd,Yes,5/27/20 0:00,3.0,Expired,36,Male,"LAPAROSCOPY, EXPLORATORY",5.0,Moribund,289.59,Closed splenic rupture,AN Post-op Complications
3,001353f9e953c8cd,2020-09-18 09:20:00,2020-09-25 16:29:00,0.000000,fc47d3637476c87e,Yes,9/20/20 0:00,15.0,Home Routine,78,Female,CABG (CORONARY ARTERY BYPASS GRAFT),4.0,Incapacitating Disease,794.39|414.00,Abnormal stress echo|Coronary artery disease,AN Post-op Complications
4,0043f49c66d1a29a,2020-04-12 11:04:00,2020-05-01 15:11:00,0.000000,558d26139a0cdfc0,Yes,4/22/20 0:00,15.0,Home Routine,64,Male,"HIPEC - CHEMOTHERAPY, INTRAPERITONEAL, HYPERTH...",3.0,Severe Systemic Disease,nan,"Malignant neoplasm of stomach, unspecified loc...",AN Post-op Complications
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154,ff6f438387a2f822,2018-11-06 15:54:00,2018-11-22 15:40:00,0.535438,2158919443f9f462,Yes,11/13/18 0:00,15.0,Home Routine,60,Male,"REPLACEMENT, AORTIC VALVE",4.0,Incapacitating Disease,428.0|428.9,Acute on chronic congestive heart failure (CMS...,AN Post-op Complications
2155,ffa83cc8c6472255,2022-06-25 09:44:00,2022-07-28 23:35:00,0.000000,73be2a55e2002335,Yes,6/26/22 0:00,30.0,Long Term Care Facility,57,Male,"CRANIOTOMY, FOR ANEURYSM OR AVM CLIPPING, WITH...",3.0,Severe Systemic Disease,435.9|437.3|nan,Cerebral vasospasm|Aneurysm of middle cerebral...,AN Post-op Complications
2156,ffd4085849ae2443,2019-04-30 16:17:00,2019-05-11 16:44:00,0.000000,5f6e3e156c0aa0d3,Yes,5/8/19 0:00,6.0,Skilled Nursing Facility,63,Male,"ENDARTERECTOMY, CAROTID",3.0,Severe Systemic Disease,437.0|434.91|272.4|433.10|443.9|444.22|790.29|...,Intracranial atherosclerosis|Acute ischemic ri...,AN Post-op Complications
2157,ffe4b7d7fa95554d,2021-06-10 05:11:00,2021-07-15 20:40:00,0.366460,e8a53b58e0a341a1,Yes,6/10/21 0:00,3.0,Expired,85,Female,HEMIGLOSSECTOMY,3.0,Severe Systemic Disease,427.31|nan|511.9|518.81|276.4|141.9|284.19|584.9,Paroxysmal atrial fibrillation (CMS-HCC)|Acute...,AN Post-op Complications
